### Day 9

We're given a large grid of numbers and are asked to find the "low points" which are cells with a smaller value than all their adjacent (not diagonal) neighbors. We're asked to sum up 1+ the values on all these low points

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

with open('d9.txt') as file:
    puzzle_input = file.readlines()
    
vents = [x.strip() for x in puzzle_input]
vents = [[int(y) for y in x] for x in vents]
vents = np.array(vents)
vents

array([[8, 6, 5, ..., 9, 9, 9],
       [8, 7, 4, ..., 8, 7, 7],
       [9, 6, 5, ..., 9, 5, 6],
       ...,
       [8, 7, 6, ..., 1, 2, 3],
       [9, 8, 9, ..., 3, 4, 4],
       [9, 9, 6, ..., 4, 6, 7]])

Trying to think about what could be done to speed this up... I don't think it will be particularly slow to just run through every cell but seems like there should be some neat trick....

Only thing I'm thinking of is marking the adjacent cells as NOT minimums when we do find a min... but that probably requires enough other checks that it's not worth it....

In [2]:
# store the results here
mins = np.zeros(shape = vents.shape)

shifts = [(-1, 0), (0, 1), (1, 0), (0, -1)]

for i in range(vents.shape[0]):
    for j in range(vents.shape[1]):
        
        cell_value = vents[i][j]
        
        comparisons = []
        for shift in shifts:
            
            next_row = i + shift[0]
            next_col = j + shift[1]
            
            if (next_row >= 0) and (next_row < len(vents)) and (next_col >= 0) and (next_col < len(vents[0])):
                comparisons.append(vents[next_row][next_col])
                
        
        if cell_value < min(comparisons):
            mins[i][j] = cell_value + 1
        

mins.sum()       
        

439.0

### Part 2

Now we have this problem having to do with basins. We're told that each location identified previously is the drain of some basin that is defined of a contiguous region without a 9 in that place. We need to find the three largest basins.

My thought is to give them all an id, write a function to expand a basin, and then split apply.

In [3]:
ids = mins.copy().astype('int')
next_id = 1
drains = []
for i in range(len(ids)):
    for j in range(len(ids[0])):
        if ids[i][j] > 0:
            ids[i][j] = next_id
            next_id += 1
            drains.append((i, j))
ids

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 209,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])

So one thing I'm noticing here is that I set the id '9' as a value with a different meaning so perhaps we should use a different flag here for the inpassable parts:

In [4]:
ids = np.where(
    vents == 9,
    -1,
    ids
)
ids

array([[  0,   0,   0, ...,  -1,  -1,  -1],
       [  0,   0,   0, ...,   0,   0,   0],
       [ -1,   0,   0, ...,  -1,   0,   0],
       ...,
       [  0,   0,   0, ..., 209,   0,   0],
       [ -1,   0,  -1, ...,   0,   0,   0],
       [ -1,  -1,   0, ...,   0,   0,   0]])

In [5]:
# This will just return a list of the coordinates of the maps
def get_basin(value, array):
    matches = np.where(array == value)
    result = []
    for i in range(len(matches[0])):
        result.append((matches[0][i], matches[1][i]))
    return result



# This will changet the values of the ids array until none more can be changed
def mark_basin(value, array):
    
    # same list of shifts
    shifts = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    
    basin_count = 0
    
    while basin_count < len(get_basin(value, array)):
        
        basin = get_basin(value, array)
        basin_count = len(basin)        
        
        for cell in basin:
            for shift in shifts:
                next_row = cell[0] + shift[0]
                next_col = cell[1] + shift[1]

                if (next_row >= 0) and (next_row < len(array)) and (next_col >= 0) and (next_col < len(array[0])):
                    if array[next_row][next_col] != -1:
                        array[next_row][next_col] = value
    
    return array


# Now we get all the ids and run this over and over
id_values = np.unique(ids)

for next_id in id_values:
    if not next_id in [0, -1]:
        ids = mark_basin(next_id, ids)   
        #print(ids)
        
# mow we have to count up the top three



In [7]:
id_counts = ids.tolist()
id_counts = reduce(lambda x, y: x + y, id_counts)
id_counts = pd.Series(id_counts)
id_counts.value_counts()

-1      2784
 142     100
 208      99
 113      91
 135      90
        ... 
 15        2
 191       2
 166       2
 133       2
 216       2
Length: 219, dtype: int64

In [8]:
100*99*91

900900